In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA


In [ ]:
# Preprocess data if necessary
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

# Load data
file_path = r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Testing\Model_Test_Data.xlsx'
df = pd.read_excel(file_path)

In [ ]:
 #Regression model of Real Total Entries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load data
file_path = r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Testing\Test_Data_ElkDeer.xlsx'
df = pd.read_excel(file_path)

# Filter data for the years 2010-2023
df_filtered = df[(df['Year'] >= 2010) & (df['Year'] <= 2023)]

# Group by unique combinations and sum 'Real_Total_Entries'
grouped_data = df_filtered.groupby(['Item_Description', 'District', 'Year'])['Real_Total_Entries'].sum().reset_index()

# Linear Regression for each unique combination
for index, combination in grouped_data.iterrows():
    plt.figure(figsize=(12, 8))  # Create a new figure for each unique combination

    subset = grouped_data[
        (grouped_data['Item_Description'] == combination['Item_Description']) &
        (grouped_data['District'] == combination['District'])
    ]

    # Prepare the data for linear regression
    X = subset['Year'].values.reshape(-1, 1)
    y = subset['Real_Total_Entries'].values

    # Create and fit the linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Predict Real_Total_Entries using the linear regression model
    predictions = model.predict(X)

    # Plot the linear regression line on the entire dataset
    plt.plot(subset['Year'], predictions, label=f'Linear Regression - {combination["Item_Description"]} - {combination["District"]}')

    plt.scatter(subset['Year'], subset['Real_Total_Entries'], label='Actual Real_Total_Entries', alpha=0.5)
    plt.xlabel('Year')
    plt.ylabel('Real_Total_Entries')
    plt.legend()
    plt.title(f'Linear Regression for Sum of Real_Total_Entries over Time - {combination["Item_Description"]} - {combination["District"]}')

    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import numpy as np

# Load data
file_path = r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Testing\Test_Data_ElkDeer.xlsx'
df = pd.read_excel(file_path)

# Filter data for the years 2010-2023
df_filtered = df[(df['Year'] >= 2006) & (df['Year'] <= 2023)]

# Group by unique combinations and sum 'Real_Total_Entries'
grouped_data = df_filtered.groupby(['Item_Description', 'District', 'Year'])['Real_Total_Entries'].sum().reset_index()

# Leave-one-out cross-validation for Linear Regression
for index, combination in grouped_data.iterrows():
    plt.figure(figsize=(12, 8))  # Create a new figure for each unique combination

    subset = grouped_data[
        (grouped_data['Item_Description'] == combination['Item_Description']) &
        (grouped_data['District'] == combination['District'])
    ]

    # Initialize arrays to store predictions and actual values
    predictions = []
    actual_values = []

    # Iterate through each data point
    for i, (_, row) in enumerate(subset.iterrows()):
        X_train = subset.loc[subset.index != i, 'Year'].values.reshape(-1, 1)
        y_train = subset.loc[subset.index != i, 'Real_Total_Entries'].values

        X_test = np.array(row['Year']).reshape(-1, 1)
        y_test = np.array(row['Real_Total_Entries'])

        # Create and fit the linear regression model
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Predict Real_Total_Entries using the linear regression model
        prediction = model.predict(X_test)

        # Store predictions and actual values
        predictions.append(prediction[0])
        actual_values.append(y_test)

    # Calculate MAE
    mae = mean_absolute_error(actual_values, predictions)
    print(f"MAE for {combination['Item_Description']} - {combination['District']}: {mae}")

    # Plot the overall linear regression line for each unique combination
    plt.plot(subset['Year'], predictions, label=f'Linear Regression - {combination["Item_Description"]} - {combination["District"]}', linestyle='dashed', color='red')

    # Plot the actual data points
    plt.scatter(subset['Year'], actual_values, label='Actual Real_Total_Entries', alpha=0.5)
    plt.xlabel('Year')
    plt.ylabel('Real_Total_Entries')
    plt.legend()
    plt.title(f'Linear Regression for Sum of Real_Total_Entries over Time - {combination["Item_Description"]} - {combination["District"]}')
    plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Load data
file_path = r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Testing\Test_Data_ElkDeer.xlsx'
df = pd.read_excel(file_path)

# Filter data for the years 2010-2023
df_filtered = df[(df['Year'] >= 2010) & (df['Year'] <= 2023)]

# Group by unique combinations and sum 'Real_Total_Entries'
grouped_data = df_filtered.groupby(['Item_Description', 'District', 'Year'])['Real_Total_Entries'].sum().reset_index()

# Leave-one-out cross-validation for Weighted Linear Regression
for (item, district), subset in grouped_data.groupby(['Item_Description', 'District']):
    plt.figure(figsize=(12, 8))  # Create a new figure for each unique combination

    # Initialize arrays to store predictions and actual values
    predictions = []
    actual_values = []

    # Iterate through each data point
    for i, (_, row) in enumerate(subset.iterrows()):
        X_train = subset.loc[subset.index != i, 'Year'].values.reshape(-1, 1)
        y_train = subset.loc[subset.index != i, 'Real_Total_Entries'].values

        X_test = np.array(row['Year']).reshape(-1, 1)
        y_test = np.array(row['Real_Total_Entries'])

        # Create and fit the weighted linear regression model
        sample_weights = np.linspace(0.0001, 1, len(X_train))
        model = LinearRegression()
        model.fit(X_train, y_train, sample_weight=sample_weights)

        # Predict Real_Total_Entries using the weighted linear regression model
        prediction = model.predict(X_test)

        # Store predictions and actual values
        predictions.append(prediction[0])
        actual_values.append(y_test)

    # Calculate RMSE for the year 2023
    rmse_2023 = np.sqrt(mean_squared_error(actual_values[-1:], predictions[-1:]))
    print(f"RMSE for {item} - {district} in 2023: {rmse_2023}")

    # Calculate MAPE for the year 2023
    mape_2023 = np.mean(np.abs((actual_values[-1] - predictions[-1]) / actual_values[-1])) * 100
    print(f"MAPE for {item} - {district} in 2023: {mape_2023}")

    # Plot the overall weighted linear regression line for each unique combination
    plt.plot(subset['Year'], predictions, label=f'Weighted Linear Regression - {item} - {district}', linestyle='dashed', color='red')

    # Plot the actual data points
    plt.scatter(subset['Year'], actual_values, label='Actual Real_Total_Entries', alpha=0.5)
    plt.xlabel('Year')
    plt.ylabel('Real_Total_Entries')
    plt.legend()
    plt.title(f'Weighted Linear Regression for Sum of Real_Total_Entries over Time - {item} - {district}')
    plt.show()


In [ ]:
import pandas as pd

# Load data
file_path = r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Testing\Test_Data_ElkDeer.xlsx'
df = pd.read_excel(file_path)

# Convert 'Year' to datetime type
df['Year'] = pd.to_datetime(df['Year'], format='%Y')

# Filter data for the years 2010-2023
df_filtered = df[(df['Year'] >= '2006') & (df['Year'] <= '2023')]

# Group by 'Item_Description', 'District', and 'Year' and calculate the sum of 'Real_Total_Entries'
df_combined = df_filtered.groupby(['Item_Description', 'District', 'Year'])['Real_Total_Entries'].sum().reset_index()

# Save the DataFrame to a new Excel file
output_file_path =  r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Testing\Test_Data_ElkDeer_Sum.xlsx'
df_combined.to_excel(output_file_path, index=False)


In [ ]:
## Arima Model
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split
import warnings

# Load data from Excel file
df = pd.read_excel(r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Testing\Test_Data_ElkDeer_Sum.xlsx')

# Convert 'Year' column to datetime
df['Year'] = pd.to_datetime(df['Year'], format='%Y')

# Function to calculate accuracy metrics
def calculate_rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

# Suppress warnings
warnings.filterwarnings("ignore")

# Ensure there are enough data points for modeling
if len(df) > 3:  # Updated condition to ensure at least four data points

    # Group data by 'item description' and 'district'
    grouped_data = df.groupby(['Item_Description', 'District'])

    # Iterate through each group
    for (item, district), group_df in grouped_data:

        # Check if there is data for the year 2023
        if 2023 in group_df['Year'].dt.year.values:

            # Split data into training and testing sets
            train_size = int(len(group_df) * 0.8)  # 80% for training, 20% for testing
            train, test = group_df.iloc[:train_size], group_df.iloc[train_size:]

            # Check if the length of the training data is sufficient for the ARIMA order
            if len(train) > 2 + 2 + 2 + 1:  # Adjust the order as needed
                # Fit the ARIMA model to the training data
                model = ARIMA(train['Real_Total_Entries'], order=(2, 2, 4))  # Adjust the order as needed
                results = model.fit()

                # Make predictions for the test data
                predicted_values = results.predict(start=len(train), end=len(group_df)-1, typ='levels')

                # Plotting actual vs predicted values for the test set
                plt.figure(figsize=(10, 6))
                plt.plot(test['Year'], test['Real_Total_Entries'], label='Actual')
                plt.plot(test['Year'], predicted_values, label='Predicted', linestyle='dashed', color='red')
                plt.title(f"ARIMA Model - {item}, {district}")
                plt.xlabel('Year')
                plt.ylabel('Real_Total_Entries')
                plt.legend()
                plt.show()

                # Calculate RMSE for the test set
                true_values = test['Real_Total_Entries']
                rmse = calculate_rmse(true_values, predicted_values)

                # Print RMSE
                print(f"RMSE for {item}, {district}: {rmse}")
            else:
                print(f"Not enough data for modeling ARIMA for {item}, {district}.")

else:
    print("Not enough data for modeling.")

# Reset warnings to default behavior
warnings.filterwarnings("default")



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np
import warnings

# Load data from Excel file
# Replace 'your_data.xlsx' with the actual path to your Excel file
df = pd.read_excel(r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Testing\Test_Data_ElkDeer_Sum.xlsx')

# Convert 'Year' column to datetime
df['Year'] = pd.to_datetime(df['Year'], format='%Y')

# Function to calculate accuracy metrics
def calculate_rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

# Suppress warnings
warnings.filterwarnings("ignore")

# Ensure there are enough data points for modeling
if len(df) > 3:  # Updated condition to ensure at least four data points

    # Group data by 'item description' and 'district'
    grouped_data = df.groupby(['Item_Description', 'District'])

    # Iterate through each group
    for (item, district), group_df in grouped_data:

        # Check if there is data for the year 2023
        if 2023 in group_df['Year'].dt.year.values:

            # Fit the exponential smoothing model to the group
            model = ExponentialSmoothing(group_df['Real_Total_Entries'], trend='add', seasonal=None)

            # Adjust the smoothing_slope parameter to give more weight to the most recent values
            results = model.fit(smoothing_slope=0.4)  # You can experiment with different values

            # Make predictions for every year
            predicted_values = results.predict(start=0, end=len(group_df)-1)

            # Plotting actual vs predicted values
            plt.figure(figsize=(10, 6))
            plt.plot(group_df['Year'], group_df['Real_Total_Entries'], label='Actual')
            plt.plot(group_df['Year'], predicted_values, label='Predicted', linestyle='dashed', color='red')
            plt.title(f"Exponential Smoothing Model - {item}, {district}")
            plt.xlabel('Year')
            plt.ylabel('Real_Total_Entries')
            plt.legend()
            plt.show()

            # Calculate RMSE for all years
            true_values = group_df['Real_Total_Entries']
            rmse = calculate_rmse(true_values, predicted_values)

            # Print RMSE
            print(f"RMSE for {item}, {district}: {rmse}")

else:
    print("Not enough data for modeling.")

# Reset warnings to default behavior
warnings.filterwarnings("default")



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import numpy as np
import warnings
from sklearn.model_selection import train_test_split

# Load data from Excel file
# Replace 'your_data.xlsx' with the actual path to your Excel file
df = pd.read_excel(r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Testing\Test_Data_ElkDeer_Sum.xlsx')

# Convert 'Year' column to datetime
df['Year'] = pd.to_datetime(df['Year'], format='%Y')

# Function to calculate accuracy metrics
def calculate_rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

# Suppress warnings
warnings.filterwarnings("ignore")

# Ensure there are enough data points for modeling
if len(df) > 3:  # Updated condition to ensure at least four data points

    # Group data by 'item description' and 'district'
    grouped_data = df.groupby(['Item_Description', 'District'])

    # Iterate through each group
    for (item, district), group_df in grouped_data:

        # Check if there is data for the year 2023 and if there are enough data points for training
        if 2023 in group_df['Year'].dt.year.values and len(group_df) > 1:

            # Split data into training and testing sets
            train_size = int(len(group_df) * 0.8)
            train, test = group_df[:train_size], group_df[train_size:]

            # Check if the training set has enough data points
            if len(train) > 1:
                # Fit the exponential smoothing model to the training data
                model = ExponentialSmoothing(train['Real_Total_Entries'], trend='add', seasonal=None)
                results = model.fit(smoothing_slope=0.7)  # You can experiment with different values

                # Make predictions for the test data
                predicted_values = results.predict(start=len(train), end=len(group_df)-1)

                # Plotting actual vs predicted values
                # Plotting actual vs predicted values
                # Plotting actual vs predicted values
                plt.figure(figsize=(10, 6))
                plt.plot(train['Year'], train['Real_Total_Entries'], label='Actual - Training Set')
                plt.plot(test['Year'], test['Real_Total_Entries'], label='Actual - Test Set')
                plt.plot(test['Year'], predicted_values, label='Predicted', linestyle='dashed', color='red')
                plt.title(f"Exponential Smoothing Model - {item}, {district}")
                plt.xlabel('Year')
                plt.ylabel('Real_Total_Entries')
                plt.legend()
                plt.show()


                # Calculate RMSE for the test set
                true_values = test['Real_Total_Entries']
                rmse = calculate_rmse(true_values, predicted_values)

                # Print RMSE
                print(f"RMSE for {item}, {district}: {rmse}")

            else:
                print(f"Not enough data in the training set for {item}, {district}.")

else:
    print("Not enough data for modeling.")

# Reset warnings to default behavior
warnings.filterwarnings("default")



In [ ]:
#Regression model MAPE
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import numpy as np

# Load data
file_path = r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Final Calculation Folder\filled_data - Copy.xlsx'  # Make sure to update the path to your actual file location
df = pd.read_excel(file_path)

# Filter data for the years 2006-2023
df_filtered = df[(df['Year'] >= 2006) & (df['Year'] <= 2023)]

# Group by unique combinations and sum 'Real_Total_Entries'
grouped_data = df_filtered.groupby(['Item_Description', 'District', 'Year'])['Real_Total_Entries'].sum().reset_index()

# Initialize a list to store MAPE values for each unique combination
mape_values = []

# Leave-one-out cross-validation for Linear Regression
for index, combination in grouped_data.iterrows():
    subset = grouped_data[
        (grouped_data['Item_Description'] == combination['Item_Description']) &
        (grouped_data['District'] == combination['District'])
    ]

    # Initialize arrays to store predictions and actual values
    predictions = []
    actual_values = []

    # Iterate through each data point
    for i, (_, row) in enumerate(subset.iterrows()):
        X_train = subset.loc[subset.index != i, 'Year'].values.reshape(-1, 1)
        y_train = subset.loc[subset.index != i, 'Real_Total_Entries'].values

        X_test = np.array(row['Year']).reshape(-1, 1)
        y_test = np.array(row['Real_Total_Entries'])

        # Create and fit the linear regression model
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Predict Real_Total_Entries using the linear regression model
        prediction = model.predict(X_test)

        # Store predictions and actual values
        predictions.append(prediction[0])
        actual_values.append(y_test)

    # Calculate MAPE for the current combination
    mape = np.mean(np.abs((np.array(actual_values) - np.array(predictions)) / np.array(actual_values))) * 100
    mape_values.append(mape)

# Calculate the total average MAPE across all unique combinations
total_average_mape = np.mean(mape_values)
print(f"Total Average MAPE: {total_average_mape}%")


In [ ]:
#Arima MAPE
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import warnings

# Load data from Excel file
df = pd.read_excel(r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Testing\Test_Data_ElkDeer_Sum.xlsx')

# Convert 'Year' column to datetime
df['Year'] = pd.to_datetime(df['Year'], format='%Y')

# Function to calculate MAPE
def calculate_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_index = y_true != 0  # Avoid division by zero
    if np.any(non_zero_index):
        return np.mean(np.abs((y_true[non_zero_index] - y_pred[non_zero_index]) / y_true[non_zero_index])) * 100
    else:
        return np.nan  # Return NaN if all y_true values are zero to avoid division by zero

# Suppress warnings
warnings.filterwarnings("ignore")

# Initialize a list to store MAPE values for each group
mape_values = []

# Ensure there are enough data points for modeling
if len(df) > 3:
    grouped_data = df.groupby(['Item_Description', 'District'])

    for (item, district), group_df in grouped_data:
        if 2023 in group_df['Year'].dt.year.values:
            train_size = int(len(group_df) * 0.8)
            train, test = group_df.iloc[:train_size], group_df.iloc[train_size:]

            if len(train) > 3:  # Ensure there's enough data
                model = ARIMA(train['Real_Total_Entries'], order=(2, 2, 4))
                results = model.fit()

                predicted_values = results.predict(start=len(train), end=len(group_df)-1, typ='levels')

                # No plotting part for calculating MAPE
                
                true_values = test['Real_Total_Entries']
                mape = calculate_mape(true_values, predicted_values)
                
                # Storing MAPE value for each combination
                mape_values.append(mape)
                print(f"MAPE for {item}, {district}: {mape}%")
            else:
                print(f"Not enough data for modeling ARIMA for {item}, {district}.")
else:
    print("Not enough data for modeling.")

# Reset warnings to default behavior
warnings.filterwarnings("default")

# Calculate the total average MAPE across all groups
if mape_values:
    total_average_mape = np.mean(mape_values)
    print(f"Total Average MAPE: {total_average_mape}%")
else:
    print("No MAPE values calculated, possibly due to insufficient data.")



In [ ]:
#Exponential Smooothing Mape
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import warnings

# Load data from Excel file
df = pd.read_excel(r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Testing\Test_Data_ElkDeer_Sum.xlsx')

# Convert 'Year' column to datetime
df['Year'] = pd.to_datetime(df['Year'], format='%Y')

# Function to calculate MAPE
def calculate_mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Avoid division by zero
    non_zero_index = y_true != 0
    return np.mean(np.abs((y_true[non_zero_index] - y_pred[non_zero_index]) / y_true[non_zero_index])) * 100

# Suppress warnings
warnings.filterwarnings("ignore")

mape_values = []  # Store MAPE values for each group

# Ensure there are enough data points for modeling
if len(df) > 3:
    grouped_data = df.groupby(['Item_Description', 'District'])

    for (item, district), group_df in grouped_data:
        if 2023 in group_df['Year'].dt.year.values:
            model = ExponentialSmoothing(group_df['Real_Total_Entries'], trend='add', seasonal=None)
            results = model.fit(smoothing_slope=0.4)  # You can experiment with different values

            predicted_values = results.predict(start=0, end=len(group_df)-1)
            true_values = group_df['Real_Total_Entries']

            # Calculate MAPE for each group
            mape = calculate_mape(true_values, predicted_values)
            mape_values.append(mape)
            print(f"MAPE for {item}, {district}: {mape}%")

else:
    print("Not enough data for modeling.")

# Reset warnings to default behavior
warnings.filterwarnings("default")

# Calculate the total average MAPE across all groups
if mape_values:
    total_average_mape = np.mean(mape_values)
    print(f"Total Average MAPE: {total_average_mape}%")
else:
    print("No MAPE values calculated, possibly due to insufficient data.")

